## Introduction
The data involves a scale model car company that operates in over 20 countries. My goal is to assist them in managing their stock, improving their marketing strategy, and detemining a budget for acquiring new customers.

To achieve this, we will address several key questions that will enable the company to make informed, data-driven decisions:

Question 1: Which products should we order more of or less of? <br>
Question 2: How should we tailor marketing and communication strategies to customer behaviors?<br>
Question 3: How much can we spend on acquiring new customers?

In [313]:
import sqlite3 
import pandas as pd
import numpy as np

In [314]:
%%capture
%load_ext sqlite3
%sql sqlite:///stores.db

### Description of Data
Customers: customer data <br>
Employees: all employee information <br>
Offices: sales office information <br>
Orders: customers' sales orders <br>
OrderDetails: sales order line for each sales order <br>
Payments: customers' payment records <br>
Products: a list of scale model cars <br>
ProductLines: a list of product line categories

### Exploring the data
The following query shows a table combining the names of each table, the attribute count, and the total number of rows in each table. I'll also print the first three rows of each individual table to examine the data.

In [315]:
conn = sqlite3.connect('stores.db')
query_table = '''
    SELECT 'Customers' AS table_name, 
           (SELECT COUNT(*)
              FROM pragma_table_info('customers')) AS number_of_attributes, 
           (SELECT COUNT(*)
              FROM customers) AS row_count 
    UNION ALL
    SELECT 'Products' AS table_name, 
           (SELECT COUNT(*)
              FROM pragma_table_info('products')) AS number_of_attributes, 
           (SELECT COUNT(*)
              FROM products) AS row_count 
    UNION ALL
    SELECT 'ProductLines' AS table_name, 
           (SELECT COUNT(*)
              FROM pragma_table_info('productlines')) AS number_of_attributes, 
           (SELECT COUNT(*)
              FROM customers) AS row_count 
    UNION ALL
    SELECT 'Orders' AS table_name, 
           (SELECT COUNT(*)
              FROM pragma_table_info('orders')) AS number_of_attributes, 
           (SELECT COUNT(*)
              FROM orders) AS row_count 
    UNION ALL
    SELECT 'OrderDetails' AS table_name, 
           (SELECT COUNT(*)
              FROM pragma_table_info('orderdetails')) AS number_of_attributes, 
           (SELECT COUNT(*)
              FROM orderdetails) AS row_count 
    UNION ALL
    SELECT 'Payments' AS table_name, 
           (SELECT COUNT(*)
              FROM pragma_table_info('payments')) AS number_of_attributes, 
           (SELECT COUNT(*)
              FROM payments) AS row_count 
    UNION ALL
    SELECT 'Employees' AS table_name, 
           (SELECT COUNT(*)
              FROM pragma_table_info('employees')) AS number_of_attributes, 
           (SELECT COUNT(*)
              FROM employees) AS row_count 
    UNION ALL
    SELECT 'Offices' AS table_name, 
           (SELECT COUNT(*)
              FROM pragma_table_info('offices')) AS number_of_attributes, 
           (SELECT COUNT(*)
              FROM offices) AS row_count
'''
        
data_table = pd.read_sql_query(query_table, conn)
print(data_table);

     table_name  number_of_attributes  row_count
0     Customers                    13        122
1      Products                     9        110
2  ProductLines                     4        122
3        Orders                     7        326
4  OrderDetails                     5       2996
5      Payments                     4        273
6     Employees                     8         23
7       Offices                     9          7


In [316]:
query_customers = '''
    SELECT *
      FROM Customers
     LIMIT 3
'''
data_customers = pd.read_sql_query(query_customers, conn)
print(data_customers)

   customerNumber                customerName contactLastName  \
0             103           Atelier graphique         Schmitt   
1             112          Signal Gift Stores            King   
2             114  Australian Collectors, Co.        Ferguson   

  contactFirstName         phone       addressLine1 addressLine2       city  \
0          Carine     40.32.2555     54, rue Royale         None     Nantes   
1             Jean    7025551838    8489 Strong St.         None  Las Vegas   
2            Peter  03 9520 4555  636 St Kilda Road      Level 3  Melbourne   

      state postalCode    country  salesRepEmployeeNumber  creditLimit  
0      None      44000     France                    1370        21000  
1        NV      83030        USA                    1166        71800  
2  Victoria       3004  Australia                    1611       117300  


In [317]:
query_products = '''
    SELECT *
      FROM Products
     LIMIT 3
'''
data_products = pd.read_sql_query(query_products, conn)
print(data_products)

  productCode                            productName   productLine  \
0    S10_1678  1969 Harley Davidson Ultimate Chopper   Motorcycles   
1    S10_1949               1952 Alpine Renault 1300  Classic Cars   
2    S10_2016                  1996 Moto Guzzi 1100i   Motorcycles   

  productScale             productVendor  \
0         1:10           Min Lin Diecast   
1         1:10   Classic Metal Creations   
2         1:10  Highway 66 Mini Classics   

                                  productDescription  quantityInStock  \
0  This replica features working kickstand, front...             7933   
1  Turnable front wheels; steering function; deta...             7305   
2  Official Moto Guzzi logos and insignias, saddl...             6625   

   buyPrice    MSRP  
0     48.81   95.70  
1     98.58  214.30  
2     68.99  118.94  


In [318]:
query_product_lines = '''
    SELECT *
      FROM ProductLines
     LIMIT 3
'''
data_product_lines = pd.read_sql_query(query_product_lines, conn)
print(data_product_lines)

    productLine                                    textDescription  \
0  Classic Cars  Attention car enthusiasts: Make your wildest c...   
1   Motorcycles  Our motorcycles are state of the art replicas ...   
2        Planes  Unique, diecast airplane and helicopter replic...   

  htmlDescription image  
0            None  None  
1            None  None  
2            None  None  


In [319]:
query_orders = '''
    SELECT *
      FROM Orders
     LIMIT 3
'''
data_orders = pd.read_sql_query(query_orders, conn)
print(data_orders)

   orderNumber   orderDate requiredDate shippedDate   status  \
0        10100  2003-01-06   2003-01-13  2003-01-10  Shipped   
1        10101  2003-01-09   2003-01-18  2003-01-11  Shipped   
2        10102  2003-01-10   2003-01-18  2003-01-14  Shipped   

                 comments  customerNumber  
0                    None             363  
1  Check on availability.             128  
2                    None             181  


In [320]:
query_order_details = '''
    SELECT *
      FROM OrderDetails
     LIMIT 3
'''
data_order_details = pd.read_sql_query(query_order_details, conn)
print(data_order_details)

   orderNumber productCode  quantityOrdered  priceEach  orderLineNumber
0        10100    S18_1749               30     136.00                3
1        10100    S18_2248               50      55.09                2
2        10100    S18_4409               22      75.46                4


In [321]:
query_payments = '''
    SELECT *
      FROM Payments
     LIMIT 3
'''
data_payments = pd.read_sql_query(query_payments, conn)
print(data_customers)

   customerNumber                customerName contactLastName  \
0             103           Atelier graphique         Schmitt   
1             112          Signal Gift Stores            King   
2             114  Australian Collectors, Co.        Ferguson   

  contactFirstName         phone       addressLine1 addressLine2       city  \
0          Carine     40.32.2555     54, rue Royale         None     Nantes   
1             Jean    7025551838    8489 Strong St.         None  Las Vegas   
2            Peter  03 9520 4555  636 St Kilda Road      Level 3  Melbourne   

      state postalCode    country  salesRepEmployeeNumber  creditLimit  
0      None      44000     France                    1370        21000  
1        NV      83030        USA                    1166        71800  
2  Victoria       3004  Australia                    1611       117300  


In [322]:
query_customers = '''
    SELECT *
      FROM Customers
     LIMIT 3
'''
data_customers = pd.read_sql_query(query_customers, conn)
print(data_customers)

   customerNumber                customerName contactLastName  \
0             103           Atelier graphique         Schmitt   
1             112          Signal Gift Stores            King   
2             114  Australian Collectors, Co.        Ferguson   

  contactFirstName         phone       addressLine1 addressLine2       city  \
0          Carine     40.32.2555     54, rue Royale         None     Nantes   
1             Jean    7025551838    8489 Strong St.         None  Las Vegas   
2            Peter  03 9520 4555  636 St Kilda Road      Level 3  Melbourne   

      state postalCode    country  salesRepEmployeeNumber  creditLimit  
0      None      44000     France                    1370        21000  
1        NV      83030        USA                    1166        71800  
2  Victoria       3004  Australia                    1611       117300  


In [323]:
query_employees = '''
    SELECT *
      FROM Employees
     LIMIT 3
'''
data_employees = pd.read_sql_query(query_employees, conn)
print(data_employees)

   employeeNumber   lastName firstName extension  \
0            1002     Murphy     Diane     x5800   
1            1056  Patterson      Mary     x4611   
2            1076   Firrelli      Jeff     x9273   

                            email officeCode  reportsTo      jobTitle  
0    dmurphy@classicmodelcars.com          1        NaN     President  
1  mpatterso@classicmodelcars.com          1     1002.0      VP Sales  
2  jfirrelli@classicmodelcars.com          1     1002.0  VP Marketing  


In [324]:
query_offices = '''
    SELECT *
      FROM Offices
     LIMIT 3
'''
data_offices = pd.read_sql_query(query_offices, conn)
print(data_offices)

  officeCode           city            phone          addressLine1  \
0          1  San Francisco  +1 650 219 4782     100 Market Street   
1          2         Boston  +1 215 837 0825      1550 Court Place   
2          3            NYC  +1 212 555 3000  523 East 53rd Street   

  addressLine2 state country postalCode territory  
0    Suite 300    CA     USA      94080        NA  
1    Suite 102    MA     USA      02107        NA  
2      apt. 5A    NY     USA      10022        NA  


The following query will find products with low stock, but high performace in terms of revenue to determine what product categories should be prioritized for restocking. 

In [325]:
query_low_stock = '''
WITH 
low_stock_table AS (
SELECT productCode, 
       ROUND(SUM(quantityOrdered) * 1.0/(SELECT quantityInStock
                                           FROM products
                                          WHERE orderdetails.productCode = products.productCode), 2) AS low_stock
  FROM orderdetails
 GROUP BY productCode
 ORDER BY low_stock DESC
 LIMIT 10
),

products_to_restock AS (
SELECT productCode, 
       SUM(quantityOrdered * priceEach) AS prod_perf
  FROM orderdetails
 WHERE productCode IN (SELECT productCode
                         FROM low_stock_table)
 GROUP BY productCode 
 ORDER BY prod_perf DESC
 LIMIT 10
)
    
SELECT productName, productLine
  FROM products
 WHERE productCode IN (SELECT productCode
                         FROM products_to_restock)
'''
low_stock_prod = pd.read_sql_query(query_low_stock, conn)
print(low_stock_prod)

                productName   productLine
0         1968 Ford Mustang  Classic Cars
1        1911 Ford Town Car  Vintage Cars
2    1928 Mercedes-Benz SSK  Vintage Cars
3  1960 BSA Gold Star DBD34   Motorcycles
4          1997 BMW F650 ST   Motorcycles
5  1928 Ford Phaeton Deluxe  Vintage Cars
6        2002 Yamaha YZR M1   Motorcycles
7             The Mayflower         Ships
8        F/A 18 Hornet 1/72        Planes
9                Pont Yacht         Ships


Vintage cars and motorcycles should be prioritized for restocking because among the ten products with the lowest stock, they perform best. 

Next I'll find the ten most and ten least engaged customers in order to evaluate the market and potential marketing strategies. 

In [326]:
query_VIP = '''
WITH 
profit_per_customer_table AS (
SELECT o.customerNumber, SUM(quantityOrdered * (priceEach - buyPrice)) AS profit
  FROM products p
  JOIN orderdetails od
    ON p.productCode = od.productCode
  JOIN orders o
    ON o.orderNumber = od.orderNumber
 GROUP BY o.customerNumber
)

SELECT contactLastName, contactFirstName, city, country, ppc.profit
  FROM customers c
  JOIN profit_per_customer_table ppc
    ON ppc.customerNumber = c.customerNumber
 ORDER BY ppc.profit DESC
 LIMIT 10
'''
VIP = pd.read_sql_query(query_VIP, conn)
print(VIP)

  contactLastName contactFirstName        city      country     profit
0          Freyre           Diego       Madrid        Spain  326519.66
1          Nelson            Susan  San Rafael          USA  236769.39
2           Young             Jeff         NYC          USA   72370.09
3        Ferguson            Peter   Melbourne    Australia   70311.07
4         Labrune          Janine       Nantes       France   60875.30
5       Natividad             Eric   Singapore    Singapore   60477.38
6        Ashworth           Rachel  Manchester           UK   60095.86
7          Graham             Mike  Auckland    New Zealand   60013.99
8             Lee             Kwai         NYC          USA   58669.10
9           Frick              Sue    San Jose          USA   55931.37


In [327]:
query_least_engaged = '''
WITH 
profit_per_customer_table AS (
SELECT o.customerNumber, SUM(quantityOrdered * (priceEach - buyPrice)) AS profit
  FROM products p
  JOIN orderdetails od
    ON p.productCode = od.productCode
  JOIN orders o
    ON o.orderNumber = od.orderNumber
 GROUP BY o.customerNumber
)

SELECT contactLastName, contactFirstName, city, country, ppc.profit
  FROM customers c
  JOIN profit_per_customer_table ppc
    ON ppc.customerNumber = c.customerNumber
 ORDER BY ppc.profit ASC
 LIMIT 10
'''
least_engaged = pd.read_sql_query(query_least_engaged, conn)
print(least_engaged)

  contactLastName contactFirstName         city  country    profit
0           Young             Mary     Glendale      USA   2610.87
1          Taylor           Leslie   Brickhaven      USA   6586.02
2         Ricotti           Franco        Milan    Italy   9532.93
3         Schmitt          Carine        Nantes   France  10063.80
4           Smith          Thomas        London       UK  10868.04
5        Cartrain         Pascale     Charleroi  Belgium  11693.99
6     Donnermeyer          Michael       Munich  Germany  13033.35
7            Choi               Yu          NYC      USA  13066.02
8           Tseng            Jerry    Cambridge      USA  13734.70
9        Chandler            Brian  Los Angeles      USA  14928.37


The highest spender is from Spain, but there is decent geographic spread with the top ten spenders being from seven different countries. The only country with multiple customers in the top ten in the USA. In term of least engaged customers, customers from six different countries are in the bottom ten. Again, the USA is the only country with several customers represented. This indicates that the USA likely already represents a large share of the customer base which could be further evaluated, but marketing towards other nations may be more worthwhile to grow presence elsewhere. 

In [328]:
query_country_profit = '''
WITH profit_per_customer AS(
SELECT orders.customerNumber, ROUND(SUM(od.quantityOrdered * (od.priceEach - products.buyPrice)),2) AS profit
  FROM orders
  JOIN orderdetails od
    ON orders.orderNumber = od.orderNumber
  JOIN products 
    ON products.productCode = od.productCode
 GROUP BY orders.customerNumber
 ORDER BY profit DESC
    )

SELECT customers.country, ROUND(SUM(ppc.profit),2) As total_profit
  FROM customers 
  JOIN profit_per_customer ppc
    ON customers.customerNumber = ppc.customerNumber
 GROUP BY customers.country
 ORDER BY total_profit DESC
 LIMIT 10
'''
country_profit = pd.read_sql_query(query_country_profit, conn)
print(country_profit)

       country  total_profit
0          USA    1308815.59
1        Spain     440004.54
2       France     413016.12
3    Australia     222207.18
4  New Zealand     189506.58
5           UK     172964.26
6        Italy     144459.86
7      Finland     117239.33
8    Singapore     101782.55
9      Denmark      85629.30


As assumed, the USA does bring in the most profit. I would recommend focusing marketing efforts on the UK and Italy because their population is similar (or lower) to that of Spain and France, but brings in less than half as much profit so there is considerable room for growth. 

I'll evalute the customer lifetime value next to get an idea of how much can reasonably be spent on marketing. 

In [329]:
query_LTV = '''
WITH 
profit_per_customer_table AS (
SELECT o.customerNumber, SUM(quantityOrdered * (priceEach - buyPrice)) AS profit
  FROM products p
  JOIN orderdetails od
    ON p.productCode = od.productCode
  JOIN orders o
    ON o.orderNumber = od.orderNumber
 GROUP BY o.customerNumber
)

SELECT ROUND(AVG(ppc.profit), 2) AS ltv -- Customer Lifetime Value, average generated by customer
  FROM profit_per_customer_table ppc
'''
LTV = pd.read_sql_query(query_LTV, conn)
print(LTV)

        ltv
0  39039.59


$39,039.59 is the average profit generated per customer over their lifetime with this store. This can serve as a guideline for marketing efforts once cost for different ads is determined.  

In [330]:
conn.close()

## Conclusion

Circling back to the stakeholders questions<br>Question 1: Which products should we order more of or less of? <br>
Question 2: How should we tailor marketing and communication strategies to customer behaviors?<br>
Question 3: How much can we spend on acquiring new customers? 
Answer 1: I would recommend ordering more vintage cars and motorcycles based on the fact that they sell often and perform the best.<br>
Answer 2: Marketing and communication strategies should be focused on the UK and Italy where there is a client base, but plenty of room for growth. They should focus on vintage cars and motorcycles to align with the market interest as well as restocking priorities.<br>
Answer 3: We can spend a maximum of $39,039.59 acquiring one new customer to break even. Presumably, more than one customer would be acquired per marketing strategy, but I would love to see data from previous marketing efforts to give a more concrete answer.  
